- Default PSO parameters
- max_back=20

In [1]:
from joblib import Parallel, delayed
import multiprocessing
import random
import pickle

import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import pygmo as pg

import tengp
import symreg
from experiment_settings import nguyen7_funset, pagie_funset, keijzer_funset, korns12_funset, vlad_funset

In [2]:
TRIALS = 100
OUTPUT_FOLDER  = 'results/pso_default_20_logs/'
PARALLEL = False

In [3]:
def run_parallel(current_data):
    if not PARALLEL:
        logs = run_experiment(current_data, cost_function)
    else:
        name, (x_train, y_train, x_test, y_test), params = current_data
        print(name)

        bounds = tengp.individual.IndividualBuilder(params).create().bounds[:]

        logs = Parallel(n_jobs=num_cores)(delayed(run_experiment_instance)(_, cost_function, x_train, y_train, params, bounds)
                                            for _ in range(TRIALS))
    return logs

def run_experiment(data_item, cost_function):
    logs = []
    
    name, (x_train, y_train, x_test, y_test), params = data_item
    print(name)
    
    bounds = tengp.individual.IndividualBuilder(params).create().bounds[:]
    
    for i in range(TRIALS):
        log = run_experiment_instance(i, cost_function, x_train, y_train, params, bounds)
        logs.append(log)
    
    return logs


def run_experiment_instance(i, cost_function, x_train, y_train, params, bounds):
    print(i, end=',')
    prob = pg.problem(cost_function(np.c_[np.ones(len(x_train)), x_train], y_train, params, bounds))
    algo = pg.algorithm(pg.pso(gen=5))
    algo.set_verbosity(1)
    pop = pg.population(prob, 50)
    pop = algo.evolve(pop)
    uda = algo.extract(pg.pso)
        
    return [x[2] for x in uda.get_log()]

class cost_function:
    def __init__(self, X, Y, params, bounds):
        self.params = params
        self.bounds = bounds
        self.X = X
        self.Y = Y
    
    def fitness(self, x):      
        individual = tengp.individual.NPIndividual(list(x), self.bounds, self.params)
        
        pred = individual.transform(self.X)
        
        try:
            return [mean_squared_error(pred, self.Y)]
        except ValueError:
            return [10000000000]
        
    def get_bounds(self):
        return self.bounds

In [4]:
kw_params = {'real_valued': True, 'max_back': 20}

params_nguyen4 = tengp.Parameters(2, 1, 1, 50, nguyen7_funset, **kw_params)
params_nguyen7 = tengp.Parameters(2, 1, 1, 50, nguyen7_funset, **kw_params)
params_nguyen10 = tengp.Parameters(3, 1, 1, 50, nguyen7_funset, **kw_params)
params_pagie1  = tengp.Parameters(3, 1, 1, 50, pagie_funset, **kw_params)
params_keijzer6 = tengp.Parameters(2, 1, 1, 50, keijzer_funset, **kw_params)
params_korns = tengp.Parameters(6, 1, 1, 50, korns12_funset, **kw_params)
params_vlad = tengp.Parameters(6, 1, 1, 50, vlad_funset, **kw_params)

all_params = [params_nguyen7, params_pagie1, params_keijzer6, params_korns, params_vlad]

data = [
    ('nguyen4', symreg.get_benchmark_poly(random, 6), params_nguyen4),
    ('nguyen7', symreg.get_benchmark_nguyen7(random, None), params_nguyen7),
    ('nguyen10', symreg.get_benchmark_nguyen10(random, None), params_nguyen10),
    ('pagie1', symreg.get_benchmark_pagie1(random, None), params_pagie1),
    ('keijzer6', symreg.get_benchmark_keijzer(random, 6), params_keijzer6),
    ('korns12', symreg.get_benchmark_korns(random, 12), params_korns),
    ('vladislasleva4', symreg.get_benchmark_vladislasleva4(random, None), params_vlad)
]

num_cores = multiprocessing.cpu_count() - 1

# Korns 12

In [5]:
name, (x_train, y_train, _, _), params = data[5]
x_train = np.c_[np.ones(len(x_train)), x_train]
bounds = tengp.individual.IndividualBuilder(params).create().bounds[:]

In [6]:
%load_ext line_profiler
import cProfile

In [7]:
def run_one():
    individual = tengp.individual.IndividualBuilder(params).create()
    output = individual.transform(x_train)
    er = mean_squared_error(output, y_train)

In [8]:
run_experiment_instance(1, cost_function, x_train, y_train, params, bounds)

1,

/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:35: RuntimeWarning: invalid value encountered in sqrt
  return  np.sqrt(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:35: RuntimeWarning: invalid value encountered in sqrt
  return  np.sqrt(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:35: RuntimeWarning: invalid value encountered in sqrt
  return  np.sqrt(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:38: RuntimeWarning: overflow encountered in exp
  return np.exp(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:17: RuntimeWarning: invalid value encountered in cos
  return np.cos(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:29: RuntimeWarning: invalid value encountered in tan
  return np.tan(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_se

[10000000000.0,
 10000000000.0,
 4.959372640575236,
 1.8452295779055348,
 1.3044779631964234]

In [9]:
random.seed(2)
cProfile.run('run_experiment_instance(1, cost_function, x_train, y_train, params, bounds)', sort='cumtime')

1,

/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:35: RuntimeWarning: invalid value encountered in sqrt
  return  np.sqrt(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:11: RuntimeWarning: invalid value encountered in greater
  return np.log(x, out=np.copy(x), where=x>0)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:38: RuntimeWarning: overflow encountered in exp
  return np.exp(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:14: RuntimeWarning: invalid value encountered in sin
  return np.sin(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:17: RuntimeWarning: invalid value encountered in cos
  return np.cos(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:23: RuntimeWarning: overflow encountered in square
  return x**2
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experi

<string>:104: RuntimeWarning: invalid value encountered in add
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: overflow encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: overflow encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in subtract
<string>:104: RuntimeWarning: invalid value encountered in add
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply


<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in add
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in add
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: overflow encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: overflow encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in add
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: overflow encountered in multiply
<string>:1

<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: overflow encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in add
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: overflow encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: overflow encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: overflow encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in add
<string>:1

<string>:104: RuntimeWarning: invalid value encountered in add
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in add
<string>:104: RuntimeWarning: invalid value encountered in add
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in add
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: overflow encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<stri

<string>:104: RuntimeWarning: invalid value encountered in add
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in add
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in multiply
<string>:104: RuntimeWarning: invalid value encountered in add
<string>:104: RuntimeWarning: invalid value encountered in add


         1190325 function calls (1180373 primitive calls) in 14.008 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   14.008   14.008 {built-in method builtins.exec}
        1    0.000    0.000   14.008   14.008 <string>:1(<module>)
        1    0.060    0.060   14.008   14.008 <ipython-input-3-cd50ba6b1c55>:29(run_experiment_instance)
      300    0.012    0.000   13.603    0.045 <ipython-input-3-cd50ba6b1c55>:47(fitness)
      300    0.625    0.002   12.306    0.041 individual.py:116(transform)
     9607    2.384    0.000    9.089    0.001 individual.py:104(compute)
    19214    0.128    0.000    2.443    0.000 dispatcher.py:420(typeof_pyval)
    19214    0.204    0.000    2.315    0.000 typeof.py:22(typeof)
     2138    2.192    0.001    2.192    0.001 experiment_settings.py:25(pow3)
        1    0.005    0.005    2.094    2.094 __init__.py:328(_population_init)
    19214    0.118    0.000  

<string>:104: RuntimeWarning: invalid value encountered in multiply


In [ ]:
%lprun -f tengp.individual.NPIndividual.transform 